In [1]:
import pandas as pd


def winer(d):
    if d["DEMOCRAT"] > d["REPUBLICAN"]:
        return "DEMOCRAT"
    else:
        return "REPUBLICAN"


def democrat(d):
    if d["DEMOCRAT"] > d["REPUBLICAN"]:
        return d["Electoral"]
    else:
        return 0


def republican(d):
    if d["DEMOCRAT"] < d["REPUBLICAN"]:
        return d["Electoral"]
    else:
        return 0


def electoral(d):
    if d["Electoral Democrat"] > d["Electoral Republican"]:
        return "DEMOCRAT"
    else:
        return "REPUBLICAN"


def parse_files(presidents, votes):
    df = pd.read_csv(votes, sep="\t", index_col=False)
    vt = dict()
    for index, row in df.iterrows():
        vt[row["Code"]] = row["Votes"]
    df = pd.read_csv(presidents, sep="\t", index_col=False)
    sumr = dict()
    for index, row in df.iterrows():
        key = (
            str(row["year"])
            + ":"
            + row["state_po"]
            + ":"
            + str(vt[row["state_po"]])
            + ":"
            + row["party_simplified"]
        )
        if key in sumr:
            sumr[key] += row["candidatevotes"]
        else:
            sumr[key] = row["candidatevotes"]
    sumr = pd.DataFrame(sumr.items())
    data = pd.DataFrame(sumr[0].str.split(":", n=3, expand=True))
    data.columns = ["Year", "State", "Electoral", "Party"]
    data["Votes"] = sumr[1]
    data["Electoral"] = pd.to_numeric(data["Electoral"])
    data = data.pivot_table("Votes", ["Year", "State", "Electoral"], "Party")
    data.reset_index(drop=False, inplace=True)
    data["Winer"] = data.apply(winer, axis=1)
    data["Electoral Democrat"] = data.apply(democrat, axis=1)
    data["Electoral Republican"] = data.apply(republican, axis=1)
    data = data.fillna(0)
    return data


def create_total(full):
    tot = pd.DataFrame(
        full[
            [
                "Year",
                "Electoral Democrat",
                "Electoral Republican",
                "DEMOCRAT",
                "REPUBLICAN",
                "LIBERTARIAN",
                "OTHER",
            ]
        ]
        .groupby("Year")
        .sum()
    )
    tot["Popular"] = tot.apply(winer, axis=1)
    tot["Electoral"] = tot.apply(electoral, axis=1)
    tot["Year"] = tot.index
    return tot


full = parse_files("./1976-2020-president.tsv", "./electoral-votes.tsv")
total = create_total(full)
full.to_csv("./1976-2020-president-py.tsv", sep="\t", index=False)
total.to_csv("./1976-2020-president-total-py.tsv", sep="\t", index=False)

In [2]:
import json

import numpy as np
import pandas as pd


def parse_file(file):
    df = pd.read_csv(file, sep="\t", index_col=False)
    d = df[["DEMOCRAT", "REPUBLICAN", "LIBERTARIAN", "OTHER"]].T.to_numpy().tolist()
    s = df["Year"].to_numpy().tolist()
    v = ["DEMOCRAT", "REPUBLICAN", "LIBERTARIAN", "OTHER"]
    data = {
        "renderTo": "presidentsTotal",
        "data": {"y": {"data": d, "vars": v, "smps": s}},
        "config": {"graphType": "Stacked", "graphOrientation": "vertical"},
    }
    with open("presidentsTotal.json", "w") as outfile:
        json.dump(data, outfile)
    # print(json.dumps(data,indent=2,sort_keys=True))


parse_file("./1976-2020-president-total-py.tsv")

In [3]:
import json

from canvasxpress.canvas import CanvasXpress
from canvasxpress.config.collection import CXConfigs
from canvasxpress.config.type import CXGraphType, CXGraphTypeOptions
from canvasxpress.data.keypair import CXDictData
from canvasxpress.render.jupyter import CXNoteBook

with open("presidentsTotal.json") as json_file:
    data = json.load(json_file)


chart_options = CXConfigs()
chart_options.add(CXGraphType(CXGraphTypeOptions.Stacked)).set_param(
    "graphOrientation", "vertical"
)

chart: CanvasXpress = CanvasXpress(
    render_to=data["renderTo"], data=data["data"], config=chart_options
)

demo_nb = CXNoteBook(chart)
demo_nb.render()

In [4]:
import json

import numpy as np
import pandas as pd


def parse_file(file):
    df = pd.read_csv(file, sep="\t", index_col=False)
    d = df[["Electoral Democrat", "Electoral Republican"]].T.to_numpy().tolist()
    s = df["Year"].to_numpy().tolist()
    v = ["Electoral Democrat", "Electoral Republican"]
    data = {
        "renderTo": "presidentsElectoral",
        "data": {"y": {"data": d, "vars": v, "smps": s}},
        "config": {"graphType": "Bar", "graphOrientation": "vertical"},
    }
    with open("presidentsElectoral.json", "w") as outfile:
        json.dump(data, outfile)
    # print(json.dumps(data,indent=2,sort_keys=True))


parse_file("./1976-2020-president-total-py.tsv")

In [5]:
import json

from canvasxpress.canvas import CanvasXpress
from canvasxpress.config.collection import CXConfigs
from canvasxpress.config.type import CXGraphType, CXGraphTypeOptions
from canvasxpress.data.keypair import CXDictData
from canvasxpress.render.jupyter import CXNoteBook

with open("presidentsElectoral.json") as json_file:
    data = json.load(json_file)


chart_options = CXConfigs()
chart_options.add(CXGraphType(CXGraphTypeOptions.Bar)).set_param(
    "graphOrientation", "vertical"
)

chart: CanvasXpress = CanvasXpress(
    render_to=data["renderTo"], data=data["data"], config=chart_options
)

demo_nb = CXNoteBook(chart)
demo_nb.render()

In [6]:
import json

import numpy as np
import pandas as pd


def parse_file(file):
    df = pd.read_csv(file, sep="\t", index_col=False)
    d = df[["DEMOCRAT", "REPUBLICAN"]].T.to_numpy().tolist()
    s = df["Year"].to_numpy().tolist()
    v = ["DEMOCRAT", "REPUBLICAN"]
    data = {
        "renderTo": "presidentsPopular",
        "data": {"y": {"data": d, "vars": v, "smps": s}},
        "config": {"graphType": "Bar", "graphOrientation": "vertical"},
    }
    with open("presidentsPopular.json", "w") as outfile:
        json.dump(data, outfile)
    # print(json.dumps(data,indent=2,sort_keys=True))


parse_file("./1976-2020-president-total-py.tsv")

In [7]:
import json

from canvasxpress.canvas import CanvasXpress
from canvasxpress.config.collection import CXConfigs
from canvasxpress.config.type import CXGraphType, CXGraphTypeOptions
from canvasxpress.data.keypair import CXDictData
from canvasxpress.render.jupyter import CXNoteBook

with open("presidentsPopular.json") as json_file:
    data = json.load(json_file)


chart_options = CXConfigs()
chart_options.add(CXGraphType(CXGraphTypeOptions.Bar)).set_param(
    "graphOrientation", "vertical"
)

chart: CanvasXpress = CanvasXpress(
    render_to=data["renderTo"], data=data["data"], config=chart_options
)

demo_nb = CXNoteBook(chart)
demo_nb.render()

In [8]:
import json

import numpy as np
import pandas as pd


def parse_file(file):
    df = pd.read_csv(file, sep="\t", index_col=False)
    d = df[["DEMOCRAT", "REPUBLICAN", "LIBERTARIAN", "OTHER"]].to_numpy().tolist()
    yr = df["Year"].to_numpy().tolist()
    ed = df["Electoral Democrat"].to_numpy().tolist()
    er = df["Electoral Republican"].to_numpy().tolist()
    wn = df["Winer"].to_numpy().tolist()
    st = df["State"].to_numpy().tolist()
    el = df["Electoral"].to_numpy().tolist()
    df["Id"] = df.index
    v = df["Id"].to_numpy().tolist()
    s = ["DEMOCRAT", "REPUBLICAN", "LIBERTARIAN", "OTHER"]
    z = {
        "Year": yr,
        "Electoral Democrat": ed,
        "Electoral Republican": er,
        "Winer": wn,
        "State": st,
        "Electoral": el,
    }
    data = {
        "renderTo": "presidentsTotal",
        "data": {"y": {"data": d, "smps": s, "vars": v}, "z": z},
        "config": {
            "graphType": "Map",
            "mapProjection": "albers",
            "topoJSON": "https://www.canvasxpress.org/data/usa-albers-states.json",
            "motionBy": "Year",
            "colorBy": "Winer",
        },
    }
    with open("presidents.json", "w") as outfile:
        json.dump(data, outfile)
    # print(json.dumps(data,indent=2,sort_keys=True))


parse_file("./1976-2020-president-py.tsv")

In [9]:
import json

from canvasxpress.canvas import CanvasXpress
from canvasxpress.config.collection import CXConfigs
from canvasxpress.config.type import CXGraphType, CXGraphTypeOptions
from canvasxpress.data.keypair import CXDictData
from canvasxpress.render.jupyter import CXNoteBook

with open("presidents.json") as json_file:
    data = json.load(json_file)


chart_options = CXConfigs()
chart_options.add(CXGraphType(CXGraphTypeOptions.Map)).set_param(
    "mapId", "albersStates"
).set_param("mapProjection", "albers").set_param("motionBy", "Year").set_param(
    "colorBy", "Winer"
).set_param(
    "topoJSON", "https://www.canvasxpress.org/data/usa-albers-states.json"
)

chart: CanvasXpress = CanvasXpress(
    render_to=data["renderTo"], data=data["data"], config=chart_options
)

demo_nb = CXNoteBook(chart)
demo_nb.render()